In [4]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [7]:
# Load dataset
df = pd.read_csv('/content/dataset.csv')

# Convert timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Sort values to simulate streaming over time
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])

# Preview
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [8]:
# Model 1: Price_t+1 = Price_t + α * (Occupancy / Capacity)
def linear_price_model(df, alpha=5.0, base_price=10.0):
    all_prices = {}

    for lot, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('Timestamp')
        prices = []
        p = base_price

        for _, row in group.iterrows():
            occ = row['Occupancy']
            cap = row['Capacity']
            p = p + alpha * (occ / cap)
            prices.append(p)

        all_prices[lot] = {
            'Timestamp': group['Timestamp'].values,
            'Price': prices
        }

    return all_prices


In [9]:
# Run Model 1 and visualize
model1_results = linear_price_model(df)

def plot_model1(system_code_number):
    data = model1_results[system_code_number]
    p = figure(title=f"Model 1 Pricing: {system_code_number}",
               x_axis_label='Time', y_axis_label='Price ($)',
               x_axis_type='datetime', width=800, height=400)

    p.line(data['Timestamp'], data['Price'], line_width=2, color='blue', legend_label="Model 1")
    p.legend.location = "top_left"
    show(p)

# Example usage:
plot_model1("BHMBCCMKT01")  # Replace with any lot ID


In [10]:
model1_results

{'BHMBCCMKT01': {'Timestamp': array(['2016-10-04T07:59:00.000000000', '2016-10-04T08:25:00.000000000',
         '2016-10-04T08:59:00.000000000', ...,
         '2016-12-19T15:29:00.000000000', '2016-12-19T16:03:00.000000000',
         '2016-12-19T16:30:00.000000000'], dtype='datetime64[ns]'),
  'Price': [10.528596187175044,
   11.083188908145582,
   11.776429809358753,
   12.70363951473137,
   14.003466204506067,
   15.53726169844021,
   17.435008665511265,
   19.575389948006933,
   21.819757365684577,
   24.124783362218373,
   26.455805892547662,
   28.734835355285963,
   30.79722703639515,
   32.66031195840555,
   34.32409012131716,
   35.75389948006933,
   37.157712305026,
   38.396880415944544,
   38.864818024263435,
   39.37608318890815,
   39.991334488734836,
   40.7105719237435,
   41.69844020797227,
   42.80762564991334,
   44.09012131715771,
   45.493934142114384,
   47.03639514731369,
   48.62218370883882,
   50.13864818024263,
   51.689774696707104,
   53.197573656845755,
   